In [1]:
import argparse
import logging
import math
import os
import random
import shutil
import sys
from contextlib import nullcontext
from pathlib import Path
from dataclasses import dataclass
from typing import List

import datasets
import numpy as np
import torch
import torch.nn.functional as F
import torch.utils.checkpoint
import transformers
from accelerate import Accelerator
from accelerate.logging import get_logger
from accelerate.utils import ProjectConfiguration, set_seed
from datasets import load_dataset, load_from_disk
from huggingface_hub import create_repo, upload_folder
from packaging import version
from peft import LoraConfig
from peft.utils import get_peft_model_state_dict
from torchvision import transforms
from tqdm import tqdm
from transformers import CLIPTextModel, CLIPTokenizer

import diffusers
from diffusers import AutoencoderKL, DDPMScheduler, DiffusionPipeline, StableDiffusionPipeline, UNet2DConditionModel
from diffusers.optimization import get_scheduler
from diffusers.training_utils import cast_training_params, compute_snr
from diffusers.utils import check_min_version, convert_state_dict_to_diffusers, is_wandb_available
from diffusers.utils.hub_utils import load_or_create_model_card, populate_model_card
from diffusers.utils.import_utils import is_xformers_available
from diffusers.utils.torch_utils import is_compiled_module

/Users/faustogerman/miniforge3/envs/DL-Project/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# To make our relative library imports work
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
sys.path.append(parent_dir)

In [3]:
@dataclass
class Config:
    pretrained_model_name_or_path: str = "CompVis/stable-diffusion-v1-4"
    datasets = ["clothes_dataset", "pixelart_dataset", "photograph_dataset"]
    learning_rate: float = 1e-05
    max_train_samples: int = 1000
    num_train_epochs: int = 20
    checkpointing_steps: int = 500
    train_batch_size: int = 1
    resume_from_checkpoint: str = "latest"
    output_dir: str = "./model_downloads/mlp_blended"
    gradient_accumulation_steps: int = 1
    # This can be set to 'no', 'fp16', 'bf16' based on system config
    mixed_precision: str = None
    report_to: str = "wandb"
    seed: int = 42

    # New fields with their argparse defaults
    revision: str = None
    variant: str = None
    dataset_config_name: str = None
    train_data_dir: str = None
    image_column: str = "image"
    caption_column: str = "text"
    validation_prompt: str = None
    num_validation_images: int = 4
    validation_epochs: int = 1
    cache_dir: str = None
    resolution: int = 512
    center_crop: bool = False
    random_flip: bool = False
    num_train_epochs: int = 100
    max_train_steps: int = None
    learning_rate: float = 1e-4
    scale_lr: bool = False
    lr_scheduler: str = "constant"
    lr_warmup_steps: int = 500
    snr_gamma: float = None
    use_8bit_adam: bool = False
    allow_tf32: bool = False
    dataloader_num_workers: int = 0
    adam_beta1: float = 0.9
    adam_beta2: float = 0.999
    adam_weight_decay: float = 1e-2
    adam_epsilon: float = 1e-08
    max_grad_norm: float = 1.0
    push_to_hub: bool = False
    hub_token: str = None
    prediction_type: str = None
    hub_model_id: str = None
    logging_dir: str = "logs"
    local_rank: int = -1
    checkpoints_total_limit: int = None
    enable_xformers_memory_efficient_attention: bool = False
    noise_offset: float = 0
    rank: int = 4
    gradient_checkpointing: bool = False


CONFIG = Config()

In [4]:
if CONFIG.report_to == "wandb":
    if not is_wandb_available():
        raise ImportError(
            "Make sure to install wandb if you want to use it for logging during training.")

    import wandb

# If passed along, set the training seed now.
if CONFIG.seed is not None:
    set_seed(CONFIG.seed)

In [5]:
accelerator_project_config = ProjectConfiguration(
    project_dir=CONFIG.output_dir,
    logging_dir=CONFIG.output_dir
)

accelerator = Accelerator(
    # gradient_accumulation_steps=CONFIG.gradient_accumulation_steps,
    # mixed_precision=CONFIG.mixed_precision,
    log_with=CONFIG.report_to,
    project_config=accelerator_project_config,
)

# Disable AMP for MPS.
if torch.backends.mps.is_available():
    accelerator.native_amp = False

# Make one log on every process with the configuration for debugging.
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO,
)

logger = get_logger(__name__, log_level="INFO")
logger.info(accelerator.state, main_process_only=False)

if accelerator.is_local_main_process:
    datasets.utils.logging.set_verbosity_warning()
    transformers.utils.logging.set_verbosity_warning()
    diffusers.utils.logging.set_verbosity_info()
else:
    datasets.utils.logging.set_verbosity_error()
    transformers.utils.logging.set_verbosity_error()
    diffusers.utils.logging.set_verbosity_error()


# Handle the repository creation
if accelerator.is_main_process:
    if CONFIG.output_dir is not None:
        os.makedirs(CONFIG.output_dir, exist_ok=True)

    # We are not pushing our model to HuggingFace Hub
    # if CONFIG.push_to_hub:
    #     repo_id = create_repo(
    #         repo_id=CONFIG.hub_model_id or Path(CONFIG.output_dir).name,
    #         exist_ok=True,
    #         token=CONFIG.hub_token
    #     ).repo_id

04/17/2024 17:35:23 - INFO - __main__ - Distributed environment: NO
Num processes: 1
Process index: 0
Local process index: 0
Device: mps

Mixed precision type: no



### Load noise scheduler, tokenizer, and models.


In [6]:
import os
import torch
from torch import nn
from diffusers.loaders import LoraLoaderMixin
from models.StableDiffusion import StableDiffusion

class GatedMoELayer(nn.Module):
    def __init__(self, channels, num_experts):
        super(GatedMoELayer, self).__init__()
        self.num_experts = num_experts

        # One expert and one gate per unet
        self.gates = nn.ModuleList([
            nn.Sequential(
                nn.Conv2d(channels, channels, 33), # Go from `(batch_size, 4, 64, 64)` to `(batch_size, 4, 32, 32)`
                nn.SiLU(),
                nn.Conv2d(channels, channels, 25), # Go from `(batch_size, 4, 32, 32)` to `(batch_size, 4, 8, 8)`
                nn.SiLU(),
                nn.Flatten(),
                nn.Linear(channels*8*8, 1),
                nn.ReLU(),
            ) for _ in range(self.num_experts)
        ])

        self.experts = nn.ModuleList([
            nn.Sequential(
                nn.Conv2d(channels, channels, 1), # Preserve input shape
                nn.SiLU(),
                nn.Conv2d(channels, channels, 1), # Preserve input shape
                nn.SiLU()
            ) for _ in range(self.num_experts)
        ])

    def forward(self, latent_unets):
        # Pass the noise through each gate. Each gate output has shape `(batch_size, 1)`
        gate_outputs = [self.gates[i](noise) for i, noise in enumerate(latent_unets)]
        gate_outputs = torch.stack((*gate_outputs, ), dim=1) # shape: `(batch_size, n_experts, 1)`
        gate_outputs = F.softmax(gate_outputs, dim=1) # Softmax across experts per example
        
        # Pass the noise through each expert. Each expert output has shape same as latent
        expert_output = [self.experts[i](noise) for i, noise in enumerate(latent_unets)]
        expert_output = torch.stack((*expert_output, ), dim=1)

        # After stacking, the expert weights have shape (batch_size, n_experts, 4, 64, 64)
        # So, we flatten them here so we can multiply by each gate. New shape: (batch_size, n_experts, 4*64*64)
        expert_output = expert_output.view((latent_unets[0].shape[0], self.num_experts, -1))

        # Combine the gated experts and reshape back to latent shape
        predicted_noise = (gate_outputs * expert_output).sum(axis=1)
        predicted_noise = predicted_noise.view(latent_unets[0].shape)

        # Output shape (batch_size, 4, 64, 64). So, all UNets are combined by experts.
        return predicted_noise


class MLPBlendedUNet(StableDiffusion):
    def __init__(self, model_path: str, lora_paths: list[str]):
        super().__init__(model_path)

        # Freeze parameters of models to save more memory
        self.text_encoder.requires_grad_(False)
        self.vae.requires_grad_(False)

        # Load the default UNets
        self.unets = nn.ModuleList([
            UNet2DConditionModel.from_pretrained(
                CONFIG.pretrained_model_name_or_path,
                subfolder="unet",
                revision=CONFIG.revision,
                variant=CONFIG.variant
            )

            for _ in lora_paths
        ])

        self.num_channels_latents: int = self.unet.config.in_channels
        self.unet = None  # let it be garbage collected since we don't need the original unet any more

        # Apply lora weights to each UNet
        for u, lora_path in zip(self.unets, lora_paths):
            # load lora weights
            state_dict, network_alphas = LoraLoaderMixin.lora_state_dict(
                lora_path, weight_name='pytorch_lora_weights.safetensors')
            LoraLoaderMixin.load_lora_into_unet(
                state_dict, network_alphas=network_alphas, unet=u)

        # Freeze the unet parameters before adding adapters
        for idx, u in enumerate(self.unets):
            print(f"Preparing UNet #{idx}")
            u.requires_grad_(False)

            for param in u.parameters():
                param.requires_grad_(False)

            if accelerator.mixed_precision == "fp16":
                weight_dtype = torch.float16

            elif accelerator.mixed_precision == "bf16":
                weight_dtype = torch.bfloat16

            unet_lora_config = LoraConfig(
                r=CONFIG.rank,
                lora_alpha=CONFIG.rank,
                init_lora_weights="gaussian",
                target_modules=["to_k", "to_q", "to_v", "to_out.0"],
            )

            # Add adapter and make sure the trainable params are in float32.
            u.add_adapter(unet_lora_config)

            if CONFIG.mixed_precision == "fp16":
                # only upcast trainable parameters (LoRA) into fp32
                cast_training_params(u, dtype=torch.float32)

            if CONFIG.enable_xformers_memory_efficient_attention:
                if is_xformers_available():
                    import xformers

                    xformers_version = version.parse(xformers.__version__)
                    u.enable_xformers_memory_efficient_attention()

                    if xformers_version == version.parse("0.0.16"):
                        logger.warning(
                            "xFormers 0.0.16 cannot be used for training in some GPUs. If you observe problems during training, please update xFormers to at least 0.0.17. See https://huggingface.co/docs/diffusers/main/en/optimization/xformers for more details."
                        )
                else:
                    raise ValueError(
                        "xformers is not available. Make sure it is installed correctly")

            if CONFIG.gradient_checkpointing:
                u.enable_gradient_checkpointing()

            print("\n")

        self.moe_layer = GatedMoELayer(
            channels=4, # match channels of latent space
            num_experts=3  # Learn a gate for each unet
        )

    def predict_noise(self, latent_inputs: torch.Tensor, timesteps: int, encoded_prompt: torch.Tensor):
        unet_outputs = []

        for unet in self.unets:
            unet_output = unet(latent_inputs, timesteps, encoded_prompt, return_dict=False)[0]
            unet_outputs.append(unet_output)

        moe_output = self.moe_layer(unet_outputs)

        # perform guidance
        noise_pred_uncond, noise_pred_text = moe_output.chunk(2)
        noise_pred = noise_pred_uncond + self.guidance_scale * (noise_pred_text - noise_pred_uncond)

        return noise_pred

In [7]:
noise_scheduler: DDPMScheduler = DDPMScheduler.from_pretrained(
    CONFIG.pretrained_model_name_or_path,
    subfolder="scheduler"
)

tokenizer: CLIPTokenizer = CLIPTokenizer.from_pretrained(
    CONFIG.pretrained_model_name_or_path,
    subfolder="tokenizer",
    revision=CONFIG.revision
)

# The blended UNet
blended_unet = MLPBlendedUNet(
    model_path="CompVis/stable-diffusion-v1-4",
    lora_paths=[
        './model_downloads/clothes_finetuned_model',
        './model_downloads/pixelart_finetuned_model',
        './model_downloads/photograph_finetuned_model'
    ]
)

# For mixed precision training we cast all non-trainable weights (vae, non-lora text_encoder and non-lora unet) to half-precision
# as these weights are only used for inference, keeping weights in full precision is not required.
weight_dtype = torch.float32

# Move unet, vae and text_encoder to device and cast to weight_dtype
blended_unet.to(accelerator.device, dtype=weight_dtype)

{'dynamic_thresholding_ratio', 'rescale_betas_zero_snr', 'timestep_spacing', 'sample_max_value', 'prediction_type', 'thresholding', 'clip_sample_range', 'variance_type'} was not found in config. Values will be initialized to default values.
{'dynamic_thresholding_ratio', 'rescale_betas_zero_snr', 'timestep_spacing', 'sample_max_value', 'prediction_type', 'thresholding', 'clip_sample_range', 'variance_type'} was not found in config. Values will be initialized to default values.
{'force_upcast', 'norm_num_groups', 'latents_std', 'latents_mean'} was not found in config. Values will be initialized to default values.
{'mid_block_type', 'time_embedding_type', 'cross_attention_norm', 'projection_class_embeddings_input_dim', 'only_cross_attention', 'dual_cross_attention', 'time_embedding_act_fn', 'encoder_hid_dim_type', 'conv_out_kernel', 'dropout', 'class_embed_type', 'addition_time_embed_dim', 'attention_type', 'mid_block_only_cross_attention', 'resnet_skip_time_act', 'addition_embed_type', 

Preparing UNet #0


Preparing UNet #1


Preparing UNet #2




MLPBlendedUNet(
  (text_encoder): CLIPTextModel(
    (text_model): CLIPTextTransformer(
      (embeddings): CLIPTextEmbeddings(
        (token_embedding): Embedding(49408, 768)
        (position_embedding): Embedding(77, 768)
      )
      (encoder): CLIPEncoder(
        (layers): ModuleList(
          (0-11): 12 x CLIPEncoderLayer(
            (self_attn): CLIPAttention(
              (k_proj): Linear(in_features=768, out_features=768, bias=True)
              (v_proj): Linear(in_features=768, out_features=768, bias=True)
              (q_proj): Linear(in_features=768, out_features=768, bias=True)
              (out_proj): Linear(in_features=768, out_features=768, bias=True)
            )
            (layer_norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (mlp): CLIPMLP(
              (activation_fn): QuickGELUActivation()
              (fc1): Linear(in_features=768, out_features=3072, bias=True)
              (fc2): Linear(in_features=3072, out_features=768, 

In [8]:
# Enable TF32 for faster training on Ampere GPUs,
# cf https://pytorch.org/docs/stable/notes/cuda.html#tensorfloat-32-tf32-on-ampere-devices
if CONFIG.allow_tf32:
    torch.backends.cuda.matmul.allow_tf32 = True

if CONFIG.scale_lr:
    CONFIG.learning_rate = (
        CONFIG.learning_rate * CONFIG.gradient_accumulation_steps *
        CONFIG.train_batch_size * accelerator.num_processes
    )

### Optimizer


In [9]:
# Initialize the optimizer
if CONFIG.use_8bit_adam:
    try:
        import bitsandbytes as bnb
    except ImportError:
        raise ImportError(
            "Please install bitsandbytes to use 8-bit Adam. You can do so by running `pip install bitsandbytes`"
        )

    optimizer_cls = bnb.optim.AdamW8bit
else:
    optimizer_cls = torch.optim.AdamW

lora_layers = filter(lambda p: p.requires_grad, blended_unet.parameters())

optimizer = optimizer_cls(
    lora_layers,
    lr=CONFIG.learning_rate,
    betas=(CONFIG.adam_beta1, CONFIG.adam_beta2),
    weight_decay=CONFIG.adam_weight_decay,
    eps=CONFIG.adam_epsilon,
)

### Dataset


In [10]:
# Get the datasets: you can either provide your own training and evaluation files (see below)
# or specify a Dataset from the hub (the dataset will be downloaded automatically from the datasets Hub).

# In distributed training, the load_dataset function guarantees that only one local process can concurrently
# download the dataset.
# TODO: ALL DATASETS NEED TO BE LOADED
if CONFIG.datasets[0] is not None:
    # UPDATING THIS FROM ORIGINAL VERSION:
    # We load from disk instead of using load_dataset
    dataset = {'train': load_from_disk(CONFIG.datasets[0])}
else:
    data_files = {}

    if CONFIG.train_data_dir is not None:
        data_files["train"] = os.path.join(CONFIG.train_data_dir, "**")

    dataset = load_dataset(
        "imagefolder",
        data_files=data_files,
        cache_dir=CONFIG.cache_dir,
    )
    # See more about loading custom images at
    # https://huggingface.co/docs/datasets/v2.4.0/en/image_load#imagefolder

# Preprocessing the datasets.
# We need to tokenize inputs and targets.
column_names = dataset["train"].column_names

python(12566) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


In [11]:
DATASET_NAME_MAPPING = {
    "lambdalabs/pokemon-blip-captions": ("image", "text"),
}

# 6. Get the column names for input/target.
# TODO: ALL DATASETS NEED TO BE LOADED
dataset_columns = DATASET_NAME_MAPPING.get(CONFIG.datasets[0], None)

if CONFIG.image_column is None:
    image_column = dataset_columns[0] if dataset_columns is not None else column_names[0]
else:
    image_column = CONFIG.image_column
    if image_column not in column_names:
        raise ValueError(
            f"--image_column' value '{CONFIG.image_column}' needs to be one of: {', '.join(column_names)}"
        )
if CONFIG.caption_column is None:
    caption_column = dataset_columns[1] if dataset_columns is not None else column_names[1]
else:
    caption_column = CONFIG.caption_column
    if caption_column not in column_names:
        raise ValueError(
            f"--caption_column' value '{CONFIG.caption_column}' needs to be one of: {', '.join(column_names)}"
        )

In [12]:
# Preprocessing the datasets.
# We need to tokenize input captions and transform the images.
def tokenize_captions(examples, is_train=True):
    captions = []
    for caption in examples[caption_column]:
        if isinstance(caption, str):
            captions.append(caption)
        elif isinstance(caption, (list, np.ndarray)):
            # take a random caption if there are multiple
            captions.append(random.choice(caption) if is_train else caption[0])
        else:
            raise ValueError(
                f"Caption column `{caption_column}` should contain either strings or lists of strings."
            )
    inputs = tokenizer(
        captions, max_length=tokenizer.model_max_length, padding="max_length", truncation=True, return_tensors="pt"
    )
    return inputs.input_ids


# Preprocessing the datasets.
train_transforms = transforms.Compose(
    [
        transforms.Resize(CONFIG.resolution,
                          interpolation=transforms.InterpolationMode.BILINEAR),
        transforms.CenterCrop(
            CONFIG.resolution) if CONFIG.center_crop else transforms.RandomCrop(CONFIG.resolution),
        transforms.RandomHorizontalFlip(
        ) if CONFIG.random_flip else transforms.Lambda(lambda x: x),
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5]),
    ]
)

In [13]:
def unwrap_model(model):
    model = accelerator.unwrap_model(model)
    model = model._orig_mod if is_compiled_module(model) else model
    return model


def preprocess_train(examples):
    images = [image.convert("RGB") for image in examples[image_column]]
    examples["pixel_values"] = [train_transforms(image) for image in images]
    examples["input_ids"] = tokenize_captions(examples)
    return examples


def collate_fn(examples):
    pixel_values = torch.stack([example["pixel_values"]
                               for example in examples])
    pixel_values = pixel_values.to(
        memory_format=torch.contiguous_format).float()
    input_ids = torch.stack([example["input_ids"] for example in examples])
    return {"pixel_values": pixel_values, "input_ids": input_ids}

In [14]:
with accelerator.main_process_first():
    if CONFIG.max_train_samples is not None:
        # WE ALSO CHANGE THIS FROM ORIGINAL:
        # dataset["train"] = dataset["train"].shuffle(seed=CONFIG.seed).select(range(CONFIG.max_train_samples))
        dataset["train"] = dataset["train"].select(
            range(CONFIG.max_train_samples))
    # Set the training transforms
    train_dataset = dataset["train"].with_transform(preprocess_train)

# DataLoaders creation:
train_dataloader = torch.utils.data.DataLoader(
    train_dataset,
    shuffle=True,
    collate_fn=collate_fn,
    batch_size=CONFIG.train_batch_size,
    num_workers=CONFIG.dataloader_num_workers,
)

In [15]:
# Scheduler and math around the number of training steps.
overrode_max_train_steps = False
num_update_steps_per_epoch = math.ceil(
    len(train_dataloader) / CONFIG.gradient_accumulation_steps)
if CONFIG.max_train_steps is None:
    CONFIG.max_train_steps = CONFIG.num_train_epochs * num_update_steps_per_epoch
    overrode_max_train_steps = True

lr_scheduler = get_scheduler(
    CONFIG.lr_scheduler,
    optimizer=optimizer,
    num_warmup_steps=CONFIG.lr_warmup_steps * accelerator.num_processes,
    num_training_steps=CONFIG.max_train_steps * accelerator.num_processes,
)

In [16]:
# Prepare everything with our `accelerator`.
unet, optimizer, train_dataloader, lr_scheduler = accelerator.prepare(
    blended_unet, optimizer, train_dataloader, lr_scheduler
)

# We need to recalculate our total training steps as the size of the training dataloader may have changed.
num_update_steps_per_epoch = math.ceil(
    len(train_dataloader) / CONFIG.gradient_accumulation_steps)
if overrode_max_train_steps:
    CONFIG.max_train_steps = CONFIG.num_train_epochs * num_update_steps_per_epoch

# Afterwards we recalculate our number of training epochs
CONFIG.num_train_epochs = math.ceil(
    CONFIG.max_train_steps / num_update_steps_per_epoch)

# We need to initialize the trackers we use, and also store our configuration.
# The trackers initializes automatically on the main process.
if accelerator.is_main_process:
    accelerator.init_trackers("text2image-fine-tune", config=vars(CONFIG))

04/17/2024 17:35:53 - ERROR - wandb.jupyter - Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
python(12567) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(12568) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
wandb: Currently logged in as: faustotnc. Use `wandb login --relogin` to force relogin
python(12572) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(12573) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


### Training


In [17]:
total_batch_size = CONFIG.train_batch_size * \
    accelerator.num_processes * CONFIG.gradient_accumulation_steps

logger.info("***** Running training *****")
logger.info(f"  Num examples = {len(train_dataset)}")
logger.info(f"  Num Epochs = {CONFIG.num_train_epochs}")
logger.info(
    f"  Instantaneous batch size per device = {CONFIG.train_batch_size}")
logger.info(
    f"  Total train batch size (w. parallel, distributed & accumulation) = {total_batch_size}")
logger.info(
    f"  Gradient Accumulation steps = {CONFIG.gradient_accumulation_steps}")
logger.info(f"  Total optimization steps = {CONFIG.max_train_steps}")
global_step = 0
first_epoch = 0

# initialize variables for keeping track of this expert's representative embedding
total_embedding = 0
num_samples = 0

# Potentially load in the weights and states from a previous save
if CONFIG.resume_from_checkpoint:
    if CONFIG.resume_from_checkpoint != "latest":
        path = os.path.basename(CONFIG.resume_from_checkpoint)
    else:
        # Get the most recent checkpoint
        dirs = os.listdir(CONFIG.output_dir)
        dirs = [d for d in dirs if d.startswith("checkpoint")]
        dirs = sorted(dirs, key=lambda x: int(x.split("-")[1]))
        path = dirs[-1] if len(dirs) > 0 else None

    if path is None:
        accelerator.print(
            f"Checkpoint '{CONFIG.resume_from_checkpoint}' does not exist. Starting a new training run."
        )
        CONFIG.resume_from_checkpoint = None
        initial_global_step = 0
    else:
        accelerator.print(f"Resuming from checkpoint {path}")
        accelerator.load_state(os.path.join(CONFIG.output_dir, path))
        global_step = int(path.split("-")[1])
        checkpoint_representative_embedding = torch.load(os.path.join(
            CONFIG.output_dir, path, 'representative_embedding.pt'))
        num_samples = global_step
        # checkpoint_representative_embedding starts out as an average
        total_embedding = checkpoint_representative_embedding * num_samples

        initial_global_step = global_step
        first_epoch = global_step // num_update_steps_per_epoch
else:
    initial_global_step = 0

progress_bar = tqdm(
    range(0, CONFIG.max_train_steps),
    initial=initial_global_step,
    desc="Steps",
    # Only show the progress bar once on each machine.
    disable=not accelerator.is_local_main_process,
)

04/17/2024 17:35:56 - INFO - __main__ - ***** Running training *****
04/17/2024 17:35:56 - INFO - __main__ -   Num examples = 1000
04/17/2024 17:35:56 - INFO - __main__ -   Num Epochs = 100
04/17/2024 17:35:56 - INFO - __main__ -   Instantaneous batch size per device = 1
04/17/2024 17:35:56 - INFO - __main__ -   Total train batch size (w. parallel, distributed & accumulation) = 1
04/17/2024 17:35:56 - INFO - __main__ -   Gradient Accumulation steps = 1
04/17/2024 17:35:56 - INFO - __main__ -   Total optimization steps = 100000


Checkpoint 'latest' does not exist. Starting a new training run.


Steps:   0%|          | 0/100000 [00:00<?, ?it/s]

In [18]:
for epoch in range(first_epoch, CONFIG.num_train_epochs):
    unet.train()
    train_loss = 0.0

    for step, batch in enumerate(train_dataloader):
        with accelerator.accumulate(unet):
            # Convert images to latent space
            latents = blended_unet.vae.encode(batch["pixel_values"].to(dtype=weight_dtype))
            latents = latents.latent_dist.sample()
            latents = latents * blended_unet.vae.config.scaling_factor

            # Sample noise that we'll add to the latents
            noise = torch.randn_like(latents)
            if CONFIG.noise_offset:
                # https://www.crosslabs.org//blog/diffusion-with-offset-noise
                noise += CONFIG.noise_offset * torch.randn((latents.shape[0], latents.shape[1], 1, 1), device=latents.device)

            bsz = latents.shape[0]
            # Sample a random timestep for each image
            timesteps = torch.randint(0, noise_scheduler.config.num_train_timesteps, (bsz,), device=latents.device)
            timesteps = timesteps.long()

            # Add noise to the latents according to the noise magnitude at each timestep
            # (this is the forward diffusion process)
            noisy_latents = noise_scheduler.add_noise(latents, noise, timesteps)

            # Get the text embedding for conditioning
            encoder_hidden_states = blended_unet.text_encoder(batch["input_ids"], return_dict=False)[0]

            # representative embedding for this expert stable diffusor
            batch_embedding = encoder_hidden_states.mean(dim=0)  # Average over sequence length
            total_embedding += batch_embedding
            num_samples += batch_embedding.shape[0]

            # Get the target for loss depending on the prediction type
            if CONFIG.prediction_type is not None:
                # set prediction_type of scheduler if defined
                noise_scheduler.register_to_config(prediction_type=CONFIG.prediction_type)

            if noise_scheduler.config.prediction_type == "epsilon":
                target = noise
            elif noise_scheduler.config.prediction_type == "v_prediction":
                target = noise_scheduler.get_velocity(latents, noise, timesteps)
            else:
                raise ValueError(f"Unknown prediction type {noise_scheduler.config.prediction_type}")

            # Predict the noise residual and compute loss
            model_pred = unet.predict_noise(noisy_latents, timesteps, encoder_hidden_states)

            if CONFIG.snr_gamma is None:
                loss = F.mse_loss(model_pred.float(), target.float(), reduction="mean")
            else:
                # Compute loss-weights as per Section 3.4 of https://arxiv.org/abs/2303.09556.
                # Since we predict the noise instead of x_0, the original formulation is slightly changed.
                # This is discussed in Section 4.2 of the same paper.
                snr = compute_snr(noise_scheduler, timesteps)
                mse_loss_weights = torch.stack([snr, CONFIG.snr_gamma * torch.ones_like(timesteps)], dim=1).min(dim=1)[0]
                if noise_scheduler.config.prediction_type == "epsilon":
                    mse_loss_weights = mse_loss_weights / snr
                elif noise_scheduler.config.prediction_type == "v_prediction":
                    mse_loss_weights = mse_loss_weights / (snr + 1)

                loss = F.mse_loss(model_pred.float(), target.float(), reduction="none")
                loss = loss.mean(dim=list(range(1, len(loss.shape)))) * mse_loss_weights
                loss = loss.mean()

            # Gather the losses across all processes for logging (if we use distributed training).
            avg_loss = accelerator.gather(loss.repeat(CONFIG.train_batch_size)).mean()
            train_loss += avg_loss.item() / CONFIG.gradient_accumulation_steps

            # Backpropagate
            accelerator.backward(loss)
            if accelerator.sync_gradients:
                params_to_clip = lora_layers
                accelerator.clip_grad_norm_(params_to_clip, CONFIG.max_grad_norm)
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()

        # Checks if the accelerator has performed an optimization step behind the scenes
        if accelerator.sync_gradients:
            progress_bar.update(1)
            global_step += 1
            accelerator.log({"train_loss": train_loss}, step=global_step)
            train_loss = 0.0

            if global_step % CONFIG.checkpointing_steps == 0:
                if accelerator.is_main_process:
                    # _before_ saving state, check if this save would set us over the `checkpoints_total_limit`
                    if CONFIG.checkpoints_total_limit is not None:
                        checkpoints = os.listdir(CONFIG.output_dir)
                        checkpoints = [d for d in checkpoints if d.startswith("checkpoint")]
                        checkpoints = sorted(checkpoints, key=lambda x: int(x.split("-")[1]))

                        # before we save the new checkpoint, we need to have at _most_ `checkpoints_total_limit - 1` checkpoints
                        if len(checkpoints) >= CONFIG.checkpoints_total_limit:
                            num_to_remove = len(checkpoints) - CONFIG.checkpoints_total_limit + 1
                            removing_checkpoints = checkpoints[0:num_to_remove]

                            logger.info(f"{len(checkpoints)} checkpoints already exist, removing {len(removing_checkpoints)} checkpoints")
                            logger.info(f"removing checkpoints: {', '.join(removing_checkpoints)}")

                            for removing_checkpoint in removing_checkpoints:
                                removing_checkpoint = os.path.join(CONFIG.output_dir, removing_checkpoint)
                                shutil.rmtree(removing_checkpoint)

                    save_path = os.path.join(CONFIG.output_dir, f"checkpoint-{global_step}")
                    accelerator.save_state(save_path)
                    checkpoint_representative_embedding = total_embedding / num_samples
                    torch.save(checkpoint_representative_embedding, os.path.join(save_path, 'representative_embedding.pt'))

                    unwrapped_unet = unwrap_model(unet)
                    unet_lora_state_dict = convert_state_dict_to_diffusers(
                        get_peft_model_state_dict(unwrapped_unet)
                    )

                    StableDiffusionPipeline.save_lora_weights(
                        save_directory=save_path,
                        unet_lora_layers=unet_lora_state_dict,
                        safe_serialization=True,
                    )

                    logger.info(f"Saved state to {save_path}")

        logs = {"step_loss": loss.detach().item(), "lr": lr_scheduler.get_last_lr()[0]}
        progress_bar.set_postfix(**logs)

        if global_step >= CONFIG.max_train_steps:
            break

    if accelerator.is_main_process:
        if CONFIG.validation_prompt is not None and epoch % CONFIG.validation_epochs == 0:
            logger.info(
                f"Running validation... \n Generating {CONFIG.num_validation_images} images with prompt:"
                f" {CONFIG.validation_prompt}."
            )
            # create pipeline
            pipeline = DiffusionPipeline.from_pretrained(
                CONFIG.pretrained_model_name_or_path,
                unet=unwrap_model(unet),
                revision=CONFIG.revision,
                variant=CONFIG.variant,
                torch_dtype=weight_dtype,
            )
            pipeline = pipeline.to(accelerator.device)
            pipeline.set_progress_bar_config(disable=True)

            # run inference
            generator = torch.Generator(device=accelerator.device)
            if CONFIG.seed is not None:
                generator = generator.manual_seed(CONFIG.seed)
            images = []
            if torch.backends.mps.is_available():
                autocast_ctx = nullcontext()
            else:
                autocast_ctx = torch.autocast(accelerator.device.type)

            with autocast_ctx:
                for _ in range(CONFIG.num_validation_images):
                    images.append(
                        pipeline(
                            CONFIG.validation_prompt, num_inference_steps=30, generator=generator).images[0]
                    )

            for tracker in accelerator.trackers:
                if tracker.name == "tensorboard":
                    np_images = np.stack([np.asarray(img) for img in images])
                    tracker.writer.add_images("validation", np_images, epoch, dataformats="NHWC")
                if tracker.name == "wandb":
                    tracker.log(
                        {
                            "validation": [
                                wandb.Image(
                                    image, caption=f"{i}: {CONFIG.validation_prompt}")
                                for i, image in enumerate(images)
                            ]
                        }
                    )

            del pipeline
            torch.cuda.empty_cache()

RuntimeError: MPS backend out of memory (MPS allocated: 18.05 GB, other allocations: 78.52 MB, max allowed: 18.13 GB). Tried to allocate 20.00 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).